In [17]:
import mediapipe as mp
import cv2
import numpy as np
import math as m
import time

In [18]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [ ]:
#Variables
l = []
l2=[]
itr=0
cal = False
delay=True
delay_count=0

target_distance = 18 #meters
target_height = 1 #meters
height = 1 #meters
    
IBO = 329 #fps
draw_length = 28 #inches
arrow_weight = 382 #grains
poundage = 45.96 #pounds
string_weight = 20 #grams

rest_y = .06  #meters
rest_x = .035 #meters

cal_x = 0
cal_y = 0

#.0254
#-0.0508

velocity = (IBO + (draw_length-30)*10-string_weight/3+min(0,-(arrow_weight-5*poundage)/3)) * 0.3048 #velocity from ft to meters

cap = cv2.VideoCapture(0)

#Generate target visualization window
path = r'C:\Users\Gavin\target.png'

target = cv2.imread(path)

tar_h, tar_w, tar_c = target.shape

cv2.namedWindow("Target", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Target", int(tar_h), int(tar_w))

cv2.imshow('Target', target)

time.sleep(0)
with mp_holistic.Holistic(min_detection_confidence=.75, min_tracking_confidence=0.75, smooth_segmentation = True) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        
        #Recolor
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        #Detections
        results = holistic.process(image)
        
        # Recolor
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
        
        #Collects World Points
        pose_w =np.array([[res.x, res.y, res.z] for res in results.pose_world_landmarks.landmark]) if results.pose_world_landmarks else np.zeros((32, 3))
        depth = np.linalg.norm(pose_w[24][2]-pose_w[25][2])
        
        pose_w[15][0]=pose_w[15][0]+rest_x
        pose_w[15][1]=pose_w[15][1]-rest_y
        
        #Draws the arrow line onto the live pane
        img_h, img_w, img_c = image.shape
        pose =np.array([[res.x, res.y, res.z] for res in results.pose_landmarks.landmark]) if results.pose_landmarks else np.zeros((32, 3))
        pose[15][0]=pose[15][0]+rest_x
        pose[15][1]=pose[15][1]-rest_y
        cv2.line(image, (int(pose[20][0]*img_w), int(pose[20][1]*img_h)), (int(pose[15][0]*img_w), int(pose[15][1]*img_h)), color=(0, 255, 0), thickness = 3)
        
        #Calculates the Current Draw Length
        current_draw_length = np.linalg.norm(pose_w[20][:1]-pose_w[15][:1])
        
        #Average of points
        if itr==10:
            l.sort()
            
            #Determines if the user is a full draw, and then calibrates it
            if (l[-1]-l[0] < .03) and (l.count(0)==0) and (cal==False) and delay==False:
                draw_length = np.average(l)
                angle_z_offset = np.average(l2)
                height = np.linalg.norm(abs(pose_w[20][1])+abs(pose_w[30][1]))
                print("CALIBRATED", draw_length, height)
                cal=True
            
            #Determines if the user has released the bow
            if (draw_length < (np.average(l) - np.average(l)*.025)) and (cal==True) and delay==False:
                #Find angles of arrow
                angle = -m.atan((pose_w[15][1]-pose_w[20][1])/(pose_w[15][0]-pose_w[20][0]))
                angle_z = np.average(l2)-angle_z_offset
                #angle_z = 0
                
                #Calculate arrow impact position
                distance_w_angle = distance/m.cos(angle_z)

                velocity_x = m.cos(angle) * velocity
                velocity_y = m.sin(angle) * velocity  

                t = (distance_w_angle-pose_w[20][0])/velocity_x

                y_target = height + velocity_y * t + 1/2 * -9.80665 * t**2-target_height + cal_y
                x_target = m.tan(angle_z)*distance + cal_x
                
                print("RELEASED", angle, angle_z, distance_w_angle, x_target, y_target)
                
                center = (int(tar_w * .5)+int(x_target*tar_w/.4), int(tar_h *.5)+int(y_target*tar_h/.4))
                cv2.circle(target, center, 30, (0, 255, 0), -1)
                cv2.imshow('Target', target)
                delay=True
                
            l=[]
            itr=0
            if delay==True:
                delay_count+=1
            if delay_count==5:
                delay=False
                delay_count=0
        else:
            l.append(current_draw_length)
            l2.append(angle_z)
            itr+=1
        
        
        #Show the live pane
        cv2.imshow('Live View', cv2.flip(image, 1)) 
        
        #Check for key, if pressed, close live pane
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()